In [1]:
from tqdm import tqdm
from workflow.fastani.remove_gunc_failed_contigs_by_contamination_sp_cluster import RemoveGuncFailedContigsByContaminationSpCluster
from workflow.config import PCT_VALUES
from workflow.external.gtdb_metadata import GtdbMetadataR207

In [2]:
DF = RemoveGuncFailedContigsByContaminationSpCluster().output().read_cached()
print(DF.shape)

UNQ_GIDS = set(DF.index.get_level_values(0))

print(f'{len(UNQ_GIDS):,} failed gids')
DF.head()

(578539, 5)
35,723 failed gids


new_sp_rep      ani        af    type  same
gid             pct                                                  
GCA_000143435.1 1    GCF_001435955.1  97.3270  0.875591  sp_rep  True
                5    GCF_001435955.1  97.3270  0.875591  sp_rep  True
                15   GCF_001435955.1  97.6349  0.829228  sp_rep  True
GCA_000153745.1 5                NaN      NaN       NaN  no_ani  True
GCA_000155005.1 1    GCF_003697165.2  96.5203  0.827668  sp_rep  True

In [3]:
from workflow.external.gtdb_metadata import GtdbMetadataR207

DF_META = GtdbMetadataR207().output().read_cached()
DF_META.head()

,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,contig_count,...,gtdb_type_species_of_genus,ncbi_taxonomy,ncbi_taxonomy_unfiltered,domain,phylum,class,order,family,genus,species
gid,,,,,,,,,,,,,,,,,,,,,
GCA_000006155.2,1916,93.12,0.0,1171,g__Bacillus (UID902),324,0.0,4305660,80.178992,426,...,f,d__Bacteria;p__Firmicutes;c__Bacilli;o__Bacill...,d__Bacteria;x__Terrabacteria group;p__Firmicut...,d__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae_G,g__Bacillus_A,s__Bacillus_A anthracis
GCA_000007325.1,1,99.95,0.0,149,k__Bacteria (UID2329),89,0.0,1973459,90.754610,1,...,t,d__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,d__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,d__Bacteria,p__Fusobacteriota,c__Fusobacteriia,o__Fusobacteriales,f__Fusobacteriaceae,g__Fusobacterium,s__Fusobacterium nucleatum
GCA_000007385.1,0,99.82,0.0,481,c__Gammaproteobacteria (UID4202),276,0.0,4190634,84.805944,1,...,f,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Xanthomonadales,f__Xanthomonadaceae,g__Xanthomonas,s__Xanthomonas oryzae
GCA_000008085.1,0,73.13,0.0,149,k__Archaea (UID2),107,0.0,462902,94.299480,1,...,t,d__Archaea;p__Nanoarchaeota;c__;o__Nanoarchaea...,d__Archaea;x__DPANN group;p__Nanoarchaeota;o__...,d__Archaea,p__Nanoarchaeota,c__Nanoarchaeia,o__Nanoarchaeales,f__Nanoarchaeaceae,g__Nanoarchaeum,s__Nanoarchaeum equitans
GCA_000008885.1,0,100.00,0.0,134,k__Bacteria (UID2495),80,0.0,617456,87.831079,2,...,f,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales_A,f__Enterobacteriaceae_A,g__Wigglesworthia,s__Wigglesworthia glossinidia_A


types:
no_af = no alignment fraction <0.5
no_ani = novel species cluster
sp_rep = found species representative within ANI radius


In [5]:
df_changed = DF[DF['same'] == False]
df_changed = df_changed.reset_index()
df_changed = df_changed[df_changed['pct'].isin(PCT_VALUES)]
df_changed.head()

,gid,pct,new_sp_rep,ani,af,type,same
0,GCA_000761495.1,1,GCF_014769185.1,96.2371,0.854244,sp_rep,False
4,GCA_000761495.1,5,GCF_014769185.1,96.2490,0.848708,sp_rep,False
9,GCA_000761495.1,10,GCF_014769185.1,96.4119,0.842254,sp_rep,False
14,GCA_000761495.1,15,GCF_014769185.1,96.5073,0.846608,sp_rep,False
19,GCA_000761495.1,20,GCF_014769185.1,96.4710,0.850575,sp_rep,False


In [10]:
all_gids = set(df_changed['gid'].unique())
n_genomes = len(all_gids)
print(f'There were {n_genomes} that failed at any level')

There were 226 that failed at any level


In [31]:
from collections import defaultdict

# Get the genera for each of those
rank_to_seen = defaultdict(lambda: defaultdict(set))
for gid in all_gids:
    tax = DF_META.loc[gid, 'gtdb_taxonomy']
    for rank in tax.split(';'):
        rank_to_seen[rank[0]][rank].add(gid)


for r, rank_dict in sorted(rank_to_seen.items()):
    print(f'{r} had {len(rank_dict):,} unique ')

c had 34 unique 
d had 2 unique 
f had 103 unique 
g had 134 unique 
o had 70 unique 
p had 24 unique 
s had 206 unique 


In [22]:
# Get the breakdown of placeholder species
n_place = 0
n_latin = 0
for gid in all_gids:
    species = DF_META.loc[gid, 'species']

    if ' sp' in species or len(species.split('_')) > 3:
        n_place += 1
    else:
        n_latin += 1

print(f'Number of latin names: {n_latin}')
print(f'Number of placeholder: {n_place}')

Number of latin names: 42
Number of placeholder: 184
